In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer

from util.constrained_zonotope import ConstrainedZonotope

# CVXPY Layer (Collision loss)

In [35]:
# formulate and solve collision check LP for pair of pre-defined zonotopes
n = 2 # dimension
m = 4 # combined order of 2 zonotopes

c1 = np.zeros((2,1))
G1 = np.eye(2)
G2 = np.array([[0.5, 0.5],[-0.5, 0.5]])

A = np.hstack((G1, -G2))

z = cp.Variable((m, 1))
v = cp.Variable()
c2 = cp.Parameter((2, 1))

b = c2 - c1

constraints = [cp.pnorm(z, p=1) <= v,
               A @ z == b]
objective = cp.Minimize(v)
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

cvxpylayer = CvxpyLayer(problem, parameters=[c2], variables=[z, v])
#c2_tch = torch.randn(2, 1, requires_grad=True)
c2_tch = torch.ones(2, 1, requires_grad=True)

# solve the problem
z_opt, v_opt = cvxpylayer(c2_tch)

# compute the gradient of the sum of the solution with respect to c2
print("c2 before:\n", c2_tch.detach().numpy())
v_opt.backward()
print("gradient:\n", c2_tch.grad.detach().numpy())

c2 before:
 [[1.]
 [1.]]
gradient:
 [[1.0000002]
 [1.0000002]]


In [37]:
# wrap in loss function
def collision_loss(c2_tch):
    # formulate and solve collision check LP for pair of pre-defined zonotopes
    n = 2 # dimension
    m = 4 # combined order of 2 zonotopes

    c1 = np.zeros((2,1))
    G1 = np.eye(2)
    G2 = np.array([[0.5, 0.5],[-0.5, 0.5]])

    A = np.hstack((G1, -G2))

    z = cp.Variable((m, 1))
    v = cp.Variable()
    c2 = cp.Parameter((2, 1))

    b = c2 - c1

    constraints = [cp.pnorm(z, p=1) <= v,
                   A @ z == b]
    objective = cp.Minimize(v)
    problem = cp.Problem(objective, constraints)
    assert problem.is_dpp()

    cvxpylayer = CvxpyLayer(problem, parameters=[c2], variables=[z, v])

    # solve the problem
    z_opt, loss = cvxpylayer(c2_tch)

    return loss

# Feedforward network

In [38]:
# 3 Layer ReLU Feedforward Network
# Output: zonotope of dimension 2, order 2 (6 total parameters)

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(4, 16)  # 5*5 from image dimension
        self.fc2 = nn.Linear(16, 10)
        self.fc3 = nn.Linear(10, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=2, bias=True)
)


In [41]:
# feed an input through the network, compute loss and gradients
inpt = torch.tensor([1,2,3,4], dtype=torch.float)
out = net(inpt)
torch.reshape(out, (2,1))

tensor([[ 0.0290],
        [-0.0203]], grad_fn=<ViewBackward>)

In [45]:
# feed an input through the network, compute loss and gradients
inpt = torch.tensor([1,2,3,4], dtype=torch.float)
out = net(inpt)
loss = collision_loss(torch.reshape(out, (2,1)))
loss.backward()
print(net.fc1.weight.grad)

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0452, -0.0904, -0.1355, -0.1807],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.2450, -0.4900, -0.7351, -0.9801],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.2599,  0.5197,  0.7796,  1.0395],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0480, -0.0961, -0.1441, -0.1922],
        [-0.2972, -0.5944, -0.8917, -1.1889],
        [-0.2969, -0.5937, -0.8906, -1.1875],
        [-0.2113, -0.4226, -0.6339, -0.8453],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.2919, -0.5838, -0.8758, -1.1677],
        [ 0.0766,  0.1531,  0.2297,  0.3062]])


In [47]:
# train the network
optimizer = optim.SGD(net.parameters(), lr=0.01)

num_iters = 5

# training loop:
for i in range(num_iters):
    optimizer.zero_grad()   # zero the gradient buffers

    # generate a new zonotope (center) from the network
    out = net(inpt)

    # plot the 

    loss = collision_loss(torch.reshape(out, (2,1)))
    loss.backward()
    optimizer.step() 

In [4]:
# collision zonotope
c_obs = np.zeros((2,1)); G_obs = np.eye(2)
Z_obs = ConstrainedZonotope(c_obs, G_obs)
Z_int = Z_out.intersect(Z_obs)
print(Z_int)

center:
	[[-0.425]
	 [ 0.034]]
generators:
	[[-0.014 -0.065  0.     0.   ]
	 [-0.167  0.345  0.     0.   ]]
constraint A:
	[[-0.014 -0.065 -1.    -0.   ]
	 [-0.167  0.345 -0.    -1.   ]]
constraint b:
	[[ 0.425]
	 [-0.034]]
